# File I/O (Read + Write) and JSON  


This notebook covers:

### Writing files
- `open(..., "w")` + `f.write(...)`
- `open(..., "a")` append mode
- `f.writelines(...)`
- `print(..., file=f)`
- `"x"` mode (exclusive create)
- Binary writing `"wb"`

### Reading files (quick review)
- `read`, `readline`, `readlines`, `for line in f`
- `tell` / `seek` (state = file pointer)

### JSON (before/after)
- `json.dump` / `json.load`
- Python objects vs file contents


## 1) `open()` and file modes 

### `open(filename, mode, encoding=None)`  

- `filename` is usually a **string** like `"notes.txt"`
- `mode` tells Python what you want to do:

**Common modes**
- `"r"` read (file must exist)
- `"w"` write (overwrite if file exists; create if it doesn't)
- `"a"` append (add to end; create if it doesn't)
- `"x"` exclusive create (fail if file already exists)

**Binary modes**
- `"rb"` read bytes
- `"wb"` write bytes
- `"ab"` append bytes

### Text vs binary
- Text mode (`"r"`, `"w"`, `"a"`) reads/writes **strings** (`str`)
- Binary mode (`"rb"`, `"wb"`, `"ab"`) reads/writes **bytes** (`bytes`)

### `encoding`
If you are writing/reading **text**, specify `encoding="utf-8"`.


## 2) Closing files and the `with` statement (explained first)

### Manual close (works, but easy to forget)
```python
f = open("data.txt", "w", encoding="utf-8")
f.write("hello")
f.close()
```

### Recommended: `with open(...) as f:`
```python
with open("data.txt", "w", encoding="utf-8") as f:
    f.write("hello")
```
When the block ends, Python automatically closes the file.


# Part A — Writing Files

## 3) Writing with `open(..., "w")` + `f.write(...)` (manual close)

### Key facts about `f.write(text)`
- `text` must be a **string**
- returns the number of characters written (you usually ignore it)
- it does **not** automatically add a newline (`\n`)


In [ ]:
# Write a new file (overwrite if it already exists)

f = open("write_demo.txt", "w", encoding="utf-8")  # "w" = overwrite/write
f.write("Line 1\n")   # we manually include \n
f.write("Line 2\n")
f.close()  # IMPORTANT when not using 'with'

print("Wrote write_demo.txt (manual close).")


## 4) Same idea, but using `with open(...)` (recommended)

This is the *same* operation, but safer because it auto-closes the file.


In [ ]:
with open("write_demo_with.txt", "w", encoding="utf-8") as f:
    f.write("Alpha\n")
    f.write("Beta\n")

print("Wrote write_demo_with.txt (with-block).")


## 5) Append mode: `open(..., "a")`

### What append mode does
- If file exists: writes are added to the **end**
- If file does not exist: it is created
- The file pointer starts at the end

This is how you “add a log line” without overwriting the file.


In [ ]:
# Start fresh so the example is predictable
with open("append_demo.txt", "w", encoding="utf-8") as f:
    f.write("Start\n")

# Append two new lines
with open("append_demo.txt", "a", encoding="utf-8") as f:
    f.write("Appended 1\n")
    f.write("Appended 2\n")

print("append_demo.txt now contains:")
with open("append_demo.txt", "r", encoding="utf-8") as f:
    print(f.read())


## 6) Writing multiple lines: `f.writelines(list_of_strings)`

### Important detail
`writelines()` does **not** add newlines automatically.

So each string should usually end with `\n`.


In [ ]:
lines = ["one\n", "two\n", "three\n"]

with open("writelines_demo.txt", "w", encoding="utf-8") as f:
    f.writelines(lines)

print("writelines_demo.txt contains:")
with open("writelines_demo.txt", "r", encoding="utf-8") as f:
    print(f.read())


## 7) Writing using `print(..., file=f)`

You already know `print(...)` prints to the screen.

But `print` has an optional parameter:
- `file=` → where to print

So this prints into a file instead of the console.

### Why this is nice
- Automatically adds a newline (like normal print)
- Lets you print multiple values with spaces
- Lets you control separator and end characters


In [ ]:
with open("print_to_file_demo.txt", "w", encoding="utf-8") as f:
    print("Hello", "world", 123, file=f)        # spaces between items
    print("A,B,C", file=f)
    print("NO NEWLINE HERE", end="", file=f)    # end="" prevents newline
    print(" <- stuck on same line", file=f)

print("print_to_file_demo.txt contains:")
with open("print_to_file_demo.txt", "r", encoding="utf-8") as f:
    print(f.read())


## 8) Exclusive create mode: `open(..., "x")`

### What `"x"` means
- Create a new file
- **Fail** if it already exists (raises `FileExistsError`)

This is useful when you want to avoid accidentally overwriting something.


In [ ]:
# We'll demonstrate safely by using try/except.
# If the file already exists, we show the error message.

filename = "exclusive_create_demo.txt"

try:
    with open(filename, "x", encoding="utf-8") as f:
        f.write("Created with mode 'x'\n")
    print("Created", filename)
except FileExistsError:
    print("Could not create:", filename, "(it already exists)")

# Show file contents if it exists
try:
    with open(filename, "r", encoding="utf-8") as f:
        print("Current file content:")
        print(f.read())
except FileNotFoundError:
    print("File does not exist yet.")


## 9) Binary writing: `open(..., "wb")`

Text mode writes `str` (characters).  
Binary mode writes `bytes` (raw byte values 0–255).

### When you use binary mode
- images, audio, pdf files
- encrypted data
- any non-text format


In [ ]:
data = b"\x00\x01\x02HELLO\xff"  # b'' makes a bytes literal

with open("binary_demo.bin", "wb") as f:
    f.write(data)

print("Wrote binary_demo.bin")

# Read it back in binary mode to prove it worked
with open("binary_demo.bin", "rb") as f:
    back = f.read()

print("Read back:", back)
print("As ints:", list(back))


# Part B — Reading Files 

We’ll read from `append_demo.txt` to demonstrate the reading methods.


## 10) `.read()` returns one big string

In [ ]:
with open("append_demo.txt", "r", encoding="utf-8") as f:
    content = f.read()
print(repr(content))


## 11) `.readline()` reads one line at a time

In [ ]:
with open("append_demo.txt", "r", encoding="utf-8") as f:
    print("1:", repr(f.readline()))
    print("2:", repr(f.readline()))


## 12) `.readlines()` returns a list of lines

In [ ]:
with open("append_demo.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
print(lines)


## 13) Looping directly over the file (most Pythonic)

In [ ]:
with open("append_demo.txt", "r", encoding="utf-8") as f:
    for line in f:
        print("Line:", line.split())


## 14) `tell()` and `seek()` (file pointer state)

- `f.tell()` returns the current position marker.
- `f.seek(pos)` moves the pointer to a position.

In text mode, the safest rule is:
> Only seek to positions that came from `tell()`.


In [ ]:
with open("append_demo.txt", "r", encoding="utf-8") as f:
    pos = f.tell()
    first = f.read(6)

    f.seek(pos)          # go back
    first_again = f.read(6)

print("first:", repr(first))
print("first_again:", repr(first_again))


# Part C — JSON (objects vs file, before/after)

## 15) `import` 

`import json` lets us use the functions inside the standard library module named `json`.


In [ ]:
import json


## 16) Create a Python object 

This is a normal Python dictionary containing lists and dictionaries.


In [ ]:
course_obj = {
    "course": "OIM3600",
    "students": [
        {"name": "Ava", "scores": [10, 9, 10]},
        {"name": "Ben", "scores": [8, 7, 9]},
    ],
    "active": True
}

print("Python object BEFORE writing:")
print(course_obj)


## 17) `json.dump(obj, file)` 
- Converts a Python object → JSON text
- Writes the JSON text into a file

We’ll show:
- whether the file exists **before**
- the file contents **after**


In [ ]:
json_filename = "course.json"

# BEFORE: show if file exists and what it contains (if anything)
import os  # standard library (used here only to check existence)
print("Exists before write?", os.path.exists(json_filename))
if os.path.exists(json_filename):
    with open(json_filename, "r", encoding="utf-8") as f:
        print("File BEFORE write:")
        print(f.read())

# WRITE
with open(json_filename, "w", encoding="utf-8") as f:
    json.dump(course_obj, f, indent=2)

# AFTER: show file contents
with open(json_filename, "r", encoding="utf-8") as f:
    print("\nFile AFTER write:")
    print(f.read())


## 18) `json.load(file)` (explained before use)

- Reads JSON text from a file
- Converts it back into Python objects (dict/list/str/int/bool/etc.)

We’ll load into a new variable so you can compare.


In [ ]:
loaded = None
print("loaded BEFORE:", loaded)

with open("course.json", "r", encoding="utf-8") as f:
    loaded = json.load(f)

print("\nloaded AFTER:", loaded)
print("Equal to original value?", loaded == course_obj)
print("Same object identity?", loaded is course_obj)


## 19) Modify object vs modify file (before/after)

Changing `loaded` does **not** change `course.json` until you write again.


In [ ]:
# BEFORE: show file content
with open("course.json", "r", encoding="utf-8") as f:
    file_before = f.read()
print("FILE BEFORE modifying object:")
print(file_before)

# Modify Python object
loaded["students"].append({"name": "Cara", "scores": [9, 9, 10]})
loaded["active"] = False

print("\nPYTHON OBJECT AFTER modification:")
print(loaded)

# File is unchanged until we dump again
with open("course.json", "r", encoding="utf-8") as f:
    file_still = f.read()
print("\nFILE still unchanged (before rewrite):")
print(file_still)

# Rewrite file
with open("course.json", "w", encoding="utf-8") as f:
    json.dump(loaded, f, indent=2)

with open("course.json", "r", encoding="utf-8") as f:
    file_after = f.read()
print("\nFILE AFTER rewrite:")
print(file_after)


# In-class exercises (writing focus)

1. Create `notes.txt` and write 5 lines using `f.write(...)` (remember `\n`).  
2. Create `notes2.txt` and write the same 5 lines using `f.writelines(...)`.  
3. Create `notes3.txt` and write the same 5 lines using `print(..., file=f)`.  
4. Append one extra line to each file using mode `"a"`.  
5. Try mode `"x"` on a filename that already exists and observe the exception.


In [ ]:
# TODO: Do the writing exercises here.


# In-class exercises (reading focus)  

1. Read Entire File:         Return the entire contents of a file as a single string.  
2. Read First N Characters:  Use read(n) to return the first n characters of a file.
3. Count Lines:              Return the number of lines in a file.
4. Return Longest Line:      Return the longest line in the file (strip the newline before returning).
5. Count Word Occurrences:    Count how many times a target word appears in a file (case-insensitive, ignore punctuation).



In [ ]:
# TODO: Do the reading exercises here.


# In-class exercises (json focus)  

using the file `course.json`

1. Get course name: load file and return the value of key 'course'  
2. Count the number of students in the file
3. Return a list of all student names
4. Return the average score of all students
5. Add a new student (name and scores) to the json file and save it
